<a id='Top of document'></a>

# Data Wrangling

* [DB-API Playground](#db_api_playground)
    * [Display as DataFrame](#display_as_df)
* [Lesson 13: OpenStreetMap Data](#lesson_13)
    * [3. Quiz: Iterative Parsing](#quiz_iter_parsing)
    * [5. Iterating Through Ways Tags](#iterating_way_tags)
    * [6. Quiz: Tag Types](#quiz_tag_types)
    * [7. Quiz: Exploring Users](#quiz_exploring_users)
    * [10. Quiz: Improving Street Names](#quiz_improving_street_names)

[Back to top](#Top of document)
<a id='db_api_playground'></a>

# DB API - Playground

In [ ]:
import sqlite3
import os

In [ ]:
path = r'D:\sqlite_windows'

In [ ]:
data_base = 'Chinook_Sqlite.db'
data_base = os.path.join(path, data_base)

In [ ]:
db = sqlite3.connect('data_base')

In [ ]:
c = db.cursor()

In [ ]:
QUERY = 'SELECT * FROM Invoice'

In [ ]:
c.execute(QUERY)

In [ ]:
rows = c.fetchall()

In [ ]:
print(rows)

In [ ]:
for i, row in enumerate(rows):
    print(f'Row {i}: {row}')

[Back to top](#Top of document)
<a id='display_as_df'></a>

## Display as a DataFrame

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 999)
pd.set_option('expand_frame_repr', False)

In [ ]:
df = pd.DataFrame(rows)
print(df)

In [ ]:
db.close()

[Back to top](#Top of document)
<a id='lesson_13'></a>

# Lesson 13: OpenStreetMap Data

### Imports

In [2]:
import xml.etree.cElementTree as ET
import pprint
import os
from collections import defaultdict
import re

### File

In [3]:
#osm_file = open('chicago_illinois.osm', 'rb')
#osm_example = open('example.osm')
osm_portland = 'D:/PythonProjects/UDACITY/01_Data_Analyst/03_Data_Wrangling/Data Wrangling ' \
            'Project/portland_oregon_50_sample.osm'
osm_portland_full = 'D:/PythonProjects/UDACITY/01_Data_Analyst/03_Data_Wrangling/Data Wrangling ' \
            'Project/portland_oregon.osm'

### OpenStreetMap Functions

In [3]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

In [4]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type] += 1

In [5]:
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print(f'{k}: {v}')

In [6]:
def is_street_name(elem):
    return (elem.tag == 'tag') and (elem.attrib['k'] == 'addr:street')

In [7]:
def audit(osm_file):
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
        print_sorted_dict(street_types)
        
    osm_file.close()

[Back to top](#Top of document)
<a id='quiz_iter_parsing'></a>

## 3. Quiz: Iterative Parsing

Your task is to use the iterative parsing to process the map file and
find out not only what tags are there, but also how many, to get the
feeling on how much of which data you can expect to have in the map.
Fill out the count_tags function. It should return a dictionary with the 
tag name as the key and number of times this tag can be encountered in 
the map as value.

[The ElemementTree iterpase function](http://effbot.org/zone/element-iterparse.htm)

In [14]:
def count_tags(filename):
    count = 0
    tag_dict = {}
    for event, elem in ET.iterparse(filename):
        count += 1
        if elem.tag not in tag_dict:
            tag_dict[elem.tag] = 1
        else:
            tag_dict[elem.tag] += 1
        if count >= 100000:
            break
    elem.clear()
    
    #filename.close()
    return tag_dict

In [13]:
count_tags(osm_portland_full)

{'bounds': 1, 'node': 9908, 'tag': 91}

## Test

In [ ]:
def test(filename):

    tags = count_tags(filename)
    pprint.pprint(tags)
    assert tags == {'bounds': 1,
                     'member': 3,
                     'nd': 4,
                     'node': 20,
                     'osm': 1,
                     'relation': 1,
                     'tag': 7,
                     'way': 1}

In [ ]:
test(osm_example)

### Non-Iterative XML Parsing Example

In [ ]:
def count_tag_non_iterative(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    tag_dict = {}
    for child in root:
        if child.tag not in tag_dict:
            tag_dict[child.tag] = 1
        else:
            tag_dict[child.tag] += 1
            
    filename.close()
    return tag_dict
    root.clear()


In [ ]:
count_tag_non_iterative(osm_example)

### Dict Testing

In [ ]:
d = {'x': 1, 'y': 2, 'z': 3}

In [ ]:
d['x'] += 1
d['x']

In [ ]:
if 'a' not in d:
    print('"a" is not a key in d')

[Back to top](#Top of document)
<a id='iterating_way_tags'></a>

## 5. Iterating through Ways Tags

### Modified OpenStreetMap Functions

In [15]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

In [ ]:
#osm_file = open('chicago_illinois.osm', 'rb')
#osm_example = open('example.osm')
osm_example = open('portland_oregon_50_sample.osm')

In [34]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

expected = ['Street', 'Avenue', 'Boulevard', 'Drive', 'Court', 'Place', 'Square', 'Lane', 'Road',
            'Parkway', 'Commons', 'Highway', 'Loop', 'Terrace', 'Trail', 'Way', 'North', 'South',
            'West', 'East', 'Circle', 'Broadway', 'Path', 'View']

In [4]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            

In [5]:
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print(f'{k}: {v}')

In [6]:
def is_street_name(elem):
    return (elem.attrib['k'] == 'addr:street')

In [26]:
def audit(filename):
    for event, elem in ET.iterparse(filename, events=('start',)):
        if elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types))
    #print_sorted_dict(street_types)
    return(dict(street_types))
    #filename.close()

In [52]:
way_tags_audit = audit(osm_portland_full)

{'155th': {'Southwest 155th'},
 '156th': {'Southwest 156th'},
 '157th': {'Southwest 157th'},
 '158th': {'Southwest 158th'},
 '160th': {'Southwest 160th'},
 '163rd': {'Southwest 163rd'},
 '165th': {'Southwest 165th'},
 '170': {'South Highway 170'},
 '211': {'Highway 211', 'Southeast Highway 211', 'South Highway 211'},
 '212': {' Southeast Hwy 212', 'SE Highway 212', 'Southeast Highway 212'},
 '213': {'South Highway 213', 'Highway 213'},
 '224': {'Northwest Highway 224',
         'South Highway 224',
         'Southeast Highway 224',
         'Southwest Highway 224'},
 '26': {'Highway 26', 'Southeast Highway 26'},
 '4616': {'4616'},
 '47': {'Northwest Highway 47',
        'Southwest Highway 47',
        'Southwest Old Highway 47'},
 '91st': {'SW 91st'},
 '99': {'Northeast Highway 99'},
 '99E': {'South Highway 99E', 'Highway 99E'},
 '99W': {'North Highway 99W',
         'Northeast Highway 99W',
         'Southwest Old Highway 99W'},
 '99e': {'South Highway 99e'},
 'Ankeny': {'SE Ankeny'},

In [53]:
unexpected_street_names = list(map(lambda k : k, way_tags_audit ))
print(unexpected_street_names)

['Ave', 'St.', '47', '213', '224', '211', '26', 'Run', 'Touchstone', '99E', 'Preakness', 'Chantilly', '99', 'Byway', '212', 'TRL', '156th', 'Rd', 'Dr', 'Blvd.', 'Northeast', 'St', 'Ave.', 'Hwy', '91st', 'Pky', 'Ankeny', 'Skidmore', '4616', 'Landing', 'Botticelli', '170', 'Sandy', 'street', '99W', 'Pkwy', 'st.', 'Willamina', 'Douglas', 'Walk', 'Pacific', 'End', 'Point', '99e', 'Terre', 'Vista', 'GLN', 'Wheatland', 'Summac', 'Cynthia', '160th', '163rd', 'Greenway', 'Mississippi', 'Crest', 'Pointe', 'Grotto', 'Downs', 'Polonius', 'Falstaff', 'Pimlico', 'Wheatherstone', 'Woods', 'Hotspur', 'Greco', 'Curve', 'Jamaica', 'Churchill', 'Miami', 'Northbound', 'Southbound', 'Dr.', 'OR)', 'Circus', 'Crescent', 'Spinosa', 'Summit', 'Pericles', 'Cervantes', 'Ln', 'Srive', 'Fieldcrest', 'Heights', 'Brighton', 'unknown', '158th', '155th', '165th', '157th', 'Rode']


### Problem Street Names
* 'Srive': {'North Marine Srive'}
* 'Southbound': {'I5 Freeway Southbound'}
* 'Pointe': {'Cabana Pointe'}
* 'Polonius': {'Polonius'}
* 'OR)': {'US 26 (OR)'}
*

[Back to top](#Top of document)
<a id='quiz_tag_types'></a>

## 6. Quiz: Tag Types

Your task is to explore the data a bit more.
Before you process the data and add it into your database, you should check the
"k" value for each "tag" and see if there are any potential problems.

We have provided you with 3 regular expressions to check for certain patterns
in the tags. As we saw in the quiz earlier, we would like to change the data
model and expand the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}
So, we have to see if we have such tags, and if we have any tags with
problematic characters.

Please complete the function 'key_type', such that we have a count of each of
four tag categories in a dictionary:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
See the 'process_map' and 'test' functions for examples of the expected format.

In [ ]:
#osm_file = open('chicago_illinois.osm', 'rb')
#osm_example = open('example.osm')
osm_example = open('portland_oregon_50_sample.osm')

In [ ]:
import xml.etree.cElementTree as ET
import pprint
import re

In [ ]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [ ]:
def key_type(element, keys):
    if element.tag == "tag":

        x = lower.search(element.attrib['k'])
        y = lower_colon.search(element.attrib['k'])
        z = problemchars.search(element.attrib['k'])

        if x:
            keys['lower'] += 1
        elif y:
            keys['lower_colon'] += 1
        elif z:
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
        
    return keys

In [ ]:
def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    
    filename.close()
    return keys

In [ ]:
process_map(osm_example)

### Test

In [ ]:
def test(filename):
    # You can use another testfile 'map.osm' to look at your solution
    # Note that the assertion below will be incorrect then.
    # Note as well that the test function here is only used in the Test Run;
    # when you submit, your code will be checked against a different dataset.
    keys = process_map(filename)
    pprint.pprint(keys)
    assert keys == {'lower': 5, 'lower_colon': 0, 'other': 1, 'problemchars': 1}

In [ ]:
test(osm_example)

[Back to top](#Top of document)
<a id='quiz_exploring_users'></a>

## 7. Quiz: Exploring Users

Your task is to explore the data a bit more.
The first task is a fun one - find out how many unique users
have contributed to the map in this particular area!

The function process_map should return a set of unique user IDs ("uid")

In [ ]:
#osm_file = open('chicago_illinois.osm', 'rb')
#osm_example = 'example.osm'
osm_example = 'portland_oregon_50_sample.osm'

In [ ]:
import xml.etree.cElementTree as ET
import pprint
import re

In [ ]:
def get_user(element, users):

    if 'uid' in element.attrib:
        users.add(element.attrib['uid'])    
    
    return users

In [ ]:
def process_map(filename):
    filename = open(filename)
    users = set()
    for _, element in ET.iterparse(filename):
        users = get_user(element, users)
        
    filename.close()
    return users

In [ ]:
unique_users = process_map(osm_example)
print(f'Unique Users: {len(unique_users)}')
pprint.pprint(unique_users)

### Test

In [ ]:
def test(filename):

    users = process_map(filename)
    pprint.pprint(users)
    assert len(users) == 6
    
test(osm_example)

[Back to top](#Top of document)
<a id='quiz_improving_street_names'></a>

## 10. Quiz: Improving Street Names

Your task in this exercise has two steps:

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected

In [4]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

In [ ]:
#osm_file = 'chicago_illinois.osm'
#osm_example = 'example_10.osm'
osm_example = 'portland_oregon_50_sample.osm'

In [5]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ['Street', 'Avenue', 'Boulevard', 'Drive', 'Court', 'Place', 'Square', 'Lane', 'Road',
            'Parkway', 'Commons', 'Highway', 'Loop', 'Terrace', 'Trail', 'Way', 'North', 'South',
            'West', 'East', 'Circle', 'Broadway', 'Path', 'View']

In [ ]:
print(unexpected_street_names)

In [16]:
# UPDATE THIS VARIABLE
mapping = {"St": "Street", "St.": "Street", "street": "Street", "st.": "Street",
           "Ave": "Avenue", "Ave.": "Avenue", "AVE": "Avenue",
           "Rd.": "Road", "Rd": "Road", "Rode": "Road",
           "Dr": "Drive", "Dr.": "Drive", "Srive": "Drive",
           "Blvd": "Boulevard", "Ln": "Lane",
           "TRL": "Trail", "Hwy": "Highway",
           "Pky": "Parkway", "Pkwy": "Parkway",
           "GLN": "Glen", "Cir": "Circle", "Ct.": "Court",
           "NW": "Northwest", "NE": "Northeast", "SE": "Southeast", "SW": "Southwest",
           "N": "North", "N.": "North"}

In [8]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [9]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [10]:
def audit(osmfile):
    osm_file = open(osmfile, "rb")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

### Used for Testing

In [ ]:
audit(osm_example)

In [ ]:
test_street_types = defaultdict(set,
            {'Ave': {'N. Lincoln Ave', 'North Lincoln Ave'},
             'Rd.': {'Baldwin Rd.'},
             'St.': {'West Lexington St.'}})

In [ ]:
for st_type, ways in test_street_types.items():
    for name in ways:
        print(name)
        x = name.split()
        for y in x:
            if y in mapping:
                name = name.replace(y, mapping[y])
        print(name)

In [ ]:
for st_type, ways in test_street_types.items():
    for name in ways:
        abv_found = list(filter(lambda street: street in name, mapping.keys()))
    for street in abv_found:
        name = name.replace(street, mapping[street])
    print(name)
    
"""Doesn't work because the '.' is still after Street"""

### Required Function

In [13]:
def update_name(name, mapping):

    x = name.split()
    for y in x:
        if y in mapping:
            name = name.replace(y, mapping[y])

    return name

### Test

In [14]:
def test(filename):
    st_types = audit(filename)
    #assert len(st_types) == 3
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print(name, "=>", better_name)
            
            '''
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"
            '''

In [81]:
test(osm_portland_full)

{'101': {'Northwest Hoyt Street #101'},
 '102': {'Southwest Macadam Avenue #102'},
 '110': {'Southeast Grace Avenue #110'},
 '150': {'Southwest 5th Street #150'},
 '155th': {'Southwest 155th'},
 '156th': {'Southwest 156th'},
 '157th': {'Southwest 157th'},
 '158th': {'Southwest 158th'},
 '160th': {'Southwest 160th'},
 '163rd': {'Southwest 163rd'},
 '165th': {'Southwest 165th'},
 '170': {'South Highway 170'},
 '211': {'Highway 211', 'South Highway 211', 'Southeast Highway 211'},
 '212': {' Southeast Hwy 212', 'SE Highway 212', 'Southeast Highway 212'},
 '213': {'South Highway 213', 'Highway 213'},
 '224': {'Northwest Highway 224',
         'South Highway 224',
         'Southeast Highway 224',
         'Southwest Highway 224'},
 '26': {'Highway 26', 'Southeast Highway 26'},
 '4616': {'4616'},
 '4637': {'4637'},
 '47': {'Northwest Highway 47',
        'Southwest Highway 47',
        'Southwest Old Highway 47'},
 '74th': {'SW 74th'},
 '91st': {'SW 91st'},
 '97055': {'44575 Southeast Kleins

In [17]:
# uses portland_oregon_50.osm
test(osm_portland)

{'156th': {'Southwest 156th'},
 '211': {'Southeast Highway 211', 'South Highway 211'},
 '212': {'Southeast Highway 212'},
 '213': {'South Highway 213'},
 '224': {'Southeast Highway 224'},
 '26': {'Southeast Highway 26', 'Highway 26'},
 '47': {'Northwest Highway 47', 'Southwest Old Highway 47'},
 '99': {'Northeast Highway 99'},
 '99E': {'Highway 99E'},
 '99W': {'Northeast State Highway 99W'},
 'Ave': {'NE 3rd Ave', 'NE 37th Ave'},
 'Byway': {'Southwest Kings Byway'},
 'Chantilly': {'Southwest Chantilly'},
 'Circus': {'Southwest Aventine Circus'},
 'Preakness': {'Southwest Preakness'},
 'Run': {'Southwest Gordons Run'},
 'St.': {'NW Raleigh St.'},
 'TRL': {'Southeast Hittay TRL'},
 'Touchstone': {'Touchstone'}}
Northeast State Highway 99W => Northeast State Highway 99W
Southwest Aventine Circus => Southwest Aventine Circus
Southeast Highway 26 => Southeast Highway 26
Highway 26 => Highway 26
NE 3rd Ave => Northeast 3rd Avenue
NE 37th Ave => Northeast 37th Avenue
NW Raleigh St. => Northwe